In [ ]:
!pip install git+https://github.com/google/BIG-bench.git

  Cloning https://github.com/google/BIG-bench.git to /tmp/pip-req-build-hsmrm_q7
  Running command git clone --filter=blob:none --quiet https://github.com/google/BIG-bench.git /tmp/pip-req-build-hsmrm_q7
  Resolved https://github.com/google/BIG-bench.git to commit 092b196c1f8f14a54bbc62f24759d43bde46dd3b
  Preparing metadata (setup.py) ... done
Processing //tmp/pip-req-build-hsmrm_q7/bleurt/bleurt-b610120347ef22b494b6d69b4316e303f5932516.zip (from bigbench==0.0.1)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.5/164.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.3/354.3 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 10.9 MB/s eta 0:00:00


In [ ]:
!pip uninstall kfac-jax -y
!pip install git+https://github.com/google-deepmind/kfac-jax@d3643a1ad85cd34ce9ec096a64c5a44708743217

  Cloning https://github.com/google-deepmind/kfac-jax (to revision d3643a1ad85cd34ce9ec096a64c5a44708743217) to /tmp/pip-req-build-_32odvkk
  Running command git clone --filter=blob:none --quiet https://github.com/google-deepmind/kfac-jax /tmp/pip-req-build-_32odvkk
  Running command git rev-parse -q --verify 'sha^d3643a1ad85cd34ce9ec096a64c5a44708743217'
  Running command git fetch -q https://github.com/google-deepmind/kfac-jax d3643a1ad85cd34ce9ec096a64c5a44708743217
  Running command git checkout -q d3643a1ad85cd34ce9ec096a64c5a44708743217
  Resolved https://github.com/google-deepmind/kfac-jax to commit d3643a1ad85cd34ce9ec096a64c5a44708743217
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 2.9 MB/s eta 0:00:00
  Created wheel for kfac-jax: filename=kfac_jax-0.0.5-py3-none-any.whl size=145934 sha256=255d2f401615bbf684313f15236db68b29092e4d4b47490d0abacf5dfa5a480c
  Stored in directory: /root/.cache/pip/wheels/84/00/92/f7138b01929

In [ ]:
#@title Import Jax libraries

import flax.linen as nn
import jax
import jax.numpy as jnp
from jax import random

import kfac_jax

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import os
import shutil
import time

In [ ]:
#@title The functional forms

def count_params(params):
  return sum([jnp.prod(jnp.arratarget(x.shape))
              for x in jax.tree_util.tree_leaves(params)])

def bnsl_dim(x, n_breaks, name):
  #out = nn.Dense(1)(x) + nn.Dense(1, use_bias=False)(nn.softplus(nn.Dense(n_breaks)(x)))
  out = nn.Dense(1)(x) + nn.Dense(1, use_bias=False)(nn.softplus(type(name, (nn.Dense,), {})(n_breaks)(x)))
  return out

class FunctionalForm(nn.Module):
  """BNSL operating in log-log space (i.e. inputs (and children of inputs) are never explicitly raised to a power)."""
  n_breaks: int

  @nn.compact
  def __call__(self, x, batch_train):
    x = jnp.log(x)

    offset = 1e-16
    #offset = 0.0

    x_train, y_train = batch_train
    x_train, y_train = jnp.log(x_train), jnp.log(y_train + offset)

    x_mean = jnp.mean(x_train, axis=0, keepdims=True)
    x_std = jnp.std(x_train, axis=0, keepdims=True)
    x_std = jnp.where(x_std == 0., 1., x_std)
    x = (x - x_mean)/x_std

    y_mean = jnp.mean(y_train)
    u = jnp.ones_like(x[:,0]) * y_mean

    eps_2 = jnp.log(1e-20)

    eps_array = jnp.ones_like(x) * eps_2

    x0 = jnp.zeros_like(x)

    a = type("irr_ent", (nn.Dense,), {})(1, use_bias=True)(x0)

    _mbnsl = jnp.concatenate([
          bnsl_dim(x, self.n_breaks, 'x'),
          a,
          eps_array,
      ], axis=-1)

    mbnsl = jax.scipy.special.logsumexp(_mbnsl, axis=-1, keepdims=False)

    mbnsl = mbnsl + u

    return jnp.exp(mbnsl)

def functional_form__sle(params, _model, batch, batch_train):
    x, y = batch
    y_pred = _model.apply(params, x, batch_train)
    pred, target = jnp.log(y_pred), jnp.log(y)
    return jnp.square(pred - target)

def functional_form__standard_le(params, _model, batch, batch_train):
    x, y = batch
    y_pred = _model.apply(params, x, batch_train)
    pred, target = jnp.log(y_pred), jnp.log(y)

    error = (pred - target) ** 2
    err_mu = jnp.mean(error)
    std_err = jnp.sqrt(err_mu + jnp.std(error) / (len(pred)**0.5)) - jnp.sqrt(err_mu)

    return std_err


In [ ]:
import bigbench
bigbench_path = bigbench.__path__[0]
#@title Imports and Setup
from typing import Any, Dict, List, Optional

from bigbench.api import task
from bigbench.api import results

import os
import dataclasses
import json
from matplotlib import pyplot as plt
import numpy as np
import pandas
import csv
import pickle as pkl
import ipywidgets
#import random
import time

@dataclasses.dataclass
class PlotSettings:
  marker: Optional[str] = None
  palette: Optional[str] = None
  linestyle: Optional[str] = None
  label: Optional[str] = None

DEFAULT_PLOT_SETTINGS = PlotSettings()
PALETTES = ['crest', 'flare', 'rocket', 'mako', 'magma', 'viridis']
LINESTYLES = ['-', '--', '-.', ':', 'solid', 'dashed', 'dashdot', 'dotted']
MARKERS = ['.', 'o', 'x', 'd', '*', 'v', '+']


with open(os.path.join(bigbench_path,'benchmark_tasks','task_human_eval.pkl'),'rb') as f:
  eval_dicts = pkl.load(f)
  HUMAN_EVAL_DICT = eval_dicts['average']
  MAX_HUMAN_EVAL_DICT = eval_dicts['expert']

PLOT_SETTINGS = {
    'BIG-G T=0': PlotSettings(marker='d', palette='flare', linestyle='-', label='BIG-G'),
    'BIG-G sparse': PlotSettings(marker='d', palette='flare', linestyle='dashed', label='BIG-G sparse'),
    'GPT': PlotSettings(marker='o', palette='crest', linestyle='dashed', label='GPT'),
    'BIG-G T=1': PlotSettings(marker='*', palette='crest', linestyle='-.', label='BIG-G T=1')
}


try:
  shutil.rmtree("plots")
except:
  pass

os.mkdir("plots")

if __name__ == "__main__":


  data_dict = {
      "name": [],
      "rmsle_train": [],
      "rmsle_extrap": [],
      "rmsle_all": [],
      "standard_le_train": [],
      "standard_le_all": [],
  }

  def plot_task(task_data: results.TaskData,
                subtask: str,
                metric: str,
                shots: Optional[List[int]] = None,
                model_families: Optional[List[str]] = None,
                plot_settings: Optional[Dict[str, PlotSettings]] = None,
                plot_human: bool = True,
                normalize_mc_grade: bool = False,
                plot_random: bool = True,
                decreasing_score: bool = True):

    task_name = task_data.name

    if subtask not in task_data.subtasks:
      raise ValueError(f'subtask {subtask} not found')

    if metric not in task_data.data[subtask]:
      raise ValueError(f'metric {metric} not found')

    normalize_results = (metric == 'multiple_choice_grade' and
                        normalize_mc_grade and
                        task_data.preferred_metric[subtask] == 'multiple_choice_grade')

    min_score = task_data.preferred_metric_low[subtask]
    max_score = task_data.preferred_metric_high[subtask]

    normalize_fn = lambda x: 100 * (x - min_score)/(max_score - min_score)

    if model_families is None:
      model_families = sorted(task_data.data[subtask][metric].keys())

    for model_family in model_families:
      if model_family not in task_data.data[subtask][metric]:
        continue

      if shots is None:
        shots = task_data.data[subtask][metric][model_family].keys()

      model_plot_settings = plot_settings.get(model_family, DEFAULT_PLOT_SETTINGS)

      for shot in shots:

        scores = task_data.scores(subtask=subtask, metric=metric, shots=shot,
                                  model_family=model_family)
        xdata = [x.params for x in scores]
        ydata = [x.score for x in scores]

        if normalize_results:
          ydata = [normalize_fn(y) for y in ydata]

        if decreasing_score:
          if metric == ("average_log_probability" or "log_likelihood") or subtask == "high_low_game":
            ydata = [-y for y in ydata]
          else:
            ydata = [(100.0 - y)/100 for y in ydata]

        axes = plt.gca()

        xdata = [np.dtype('float64').type(_x) for _x in xdata]

        for end in [9]:

          import gc; gc.collect()

          start_time = time.time()

          x_train = jnp.array(xdata[:end])
          y_train = jnp.array(ydata[:end])

          x_test = jnp.array(xdata[end:])
          y_test = jnp.array(ydata[end:])

          x_train = jnp.expand_dims(x_train, -1)
          x_test = jnp.expand_dims(x_test, -1)

          x_all = jnp.expand_dims(jnp.array(xdata), -1)
          y_all = jnp.array(ydata)



          #n_breaks = 2
          n_breaks = 1

          model = FunctionalForm(n_breaks=n_breaks)
          params = model.init(random.PRNGKey(21), x_train, (x_train, y_train))

          def loss(params, batch, offset=1e-16):

            x, y = batch
            y_pred = model.apply(params, x, batch)

            pred, target = jnp.log(offset + y_pred), jnp.log(offset + y)

            y_mean = jnp.mean(target)
            y_std = jnp.std(target)
            y_std = jnp.where(y_std == 0., 1., y_std)

            target = target / y_std
            pred = pred / y_std

            kfac_jax.register_squared_error_loss(pred, target)

            return jnp.mean(jnp.square(pred - target))
          loss_and_grad = jax.value_and_grad(loss, argnums=0)

          # using a second order optimizer makes training way faster
          optimizer = kfac_jax.Optimizer(
            value_and_grad_func=loss_and_grad,
            l2_reg=0.0, # 1e-4,
            value_func_has_aux=False,
            value_func_has_state=False,
            value_func_has_rng=False,
            use_adaptive_learning_rate=True,
            use_adaptive_momentum=True,
            use_adaptive_damping=True,
            initial_damping=1.0,
            multi_device=False,
          )

          rng = random.PRNGKey(0)
          rng, init_rng = random.split(rng)
          opt_state = optimizer.init(params, init_rng, (x_train, y_train))

          start_time = time.time()

          # Fits in <1min on 1 GPU
          best_params, best_loss = params, 1e6


          for j in range(int(1e1)):
            params = model.init(random.PRNGKey(j), x_train, (x_train, y_train))
            rng = random.PRNGKey(j)
            rng, init_rng = random.split(rng)
            opt_state = optimizer.init(params, init_rng, (x_train, y_train))
            for i in range(int(1e3)):
              if i == 0:
                #print("j =", j)
                pass
              rng, step_rng = jax.random.split(rng)
              params, opt_state, stats = optimizer.step(
                  params, opt_state, step_rng, batch=(x_train, y_train), global_step_int=i)
              if jnp.isnan(stats['loss']):
                break
              if (stats['loss'] < best_loss) and (stats['loss'] > 1e-12):
                best_params = jax.tree_map(lambda x: jnp.array(x), params)
                best_loss = stats['loss']
                if i % 25 == 0:
                  sle_train = functional_form__sle(best_params, model, (x_train, y_train), (x_train, y_train))
                  sle_test = functional_form__sle(best_params, model, (x_test, y_test), (x_train, y_train))
                  #print(f"{jnp.sqrt(jnp.mean(sle_train)):.5e}", f"{jnp.sqrt(jnp.mean(sle_test)):.5e}", f"{best_loss:.5e}", "   ", f"{(time.time() - start_time):.5e}", "", j, i)

          bc = best_params
          print()
          title = f'{subtask}'+" "+str(model_family)+" "+str(shot)+"-shot"
          print(title)
          print(bc)
          print("time:", time.time() - start_time)

          sle_all = functional_form__sle(bc, model, (x_all, y_all), (x_train, y_train))
          sle_train = functional_form__sle(bc, model, (x_train, y_train), (x_train, y_train))
          sle_test = functional_form__sle(bc, model, (x_test, y_test), (x_train, y_train))

          standard_le_all = functional_form__standard_le(bc, model, (x_all, y_all), (x_train, y_train))
          standard_le_train = functional_form__standard_le(bc, model, (x_train, y_train), (x_train, y_train))

          print("rmsle all:   ", jnp.sqrt(jnp.mean(sle_all)))
          print("rmsle train: ", jnp.sqrt(jnp.mean(sle_train)))
          print("rmsle test:  ", jnp.sqrt(jnp.mean(sle_test)))

          print("standard_le_all:   ", standard_le_all)
          print("standard_le_train: ", standard_le_train)

          data_dict["name"].append(title)
          data_dict["rmsle_train"].append(jnp.sqrt(jnp.mean(sle_train)))
          data_dict["rmsle_extrap"].append(jnp.sqrt(jnp.mean(sle_test)))
          data_dict["rmsle_all"].append(jnp.sqrt(jnp.mean(sle_all)))
          data_dict["standard_le_train"].append(standard_le_train)
          data_dict["standard_le_all"].append(standard_le_all)

          points = 1024
          x_tile = jnp.expand_dims(jnp.logspace(-1, 15, points), -1)

          pred = model.apply(bc, x_tile, (x_train, y_train))

          plt.plot(x_train, y_train, 'o', color='black', markersize=9.1)
          plt.plot(x_test, y_test, 'o', color=[0.0, 0.835, 0.0], markersize=9.37, markerfacecolor=[0.0, 1.0, 0.0])
          plt.plot(x_tile, pred, color=[1.0, 0.1275, 0.1275], linewidth=2.15, alpha=1.0)

          plt.xlim(np.array(xdata).min()*.8,np.array(xdata).max()*1.3)
          plt.ylim(np.array(ydata).min()*.9,np.array(ydata).max()*1.05)

          axes.set_title(title, fontsize=16)
          axes.set_xlabel(f'Number of Model Parameters', fontsize=12)
          axes.set_ylabel(metric, fontsize=12)
          plt.tick_params(axis='both', labelsize=12)

          """
          plt.xscale('log')
          plt.yscale('log')
          #"""

          #"""
          axes.set_xscale('log')
          axes.set_yscale('log')
          #"""

          plt.savefig('plots/'+str(initial_task)+"__"+str(model_family)+"__"+str(shot)+"-shot"+'.png', bbox_inches='tight')

          plt.show()

          #plt.close()
          #plt.cla()
          #plt.clf()

    return None

  #@title Load Data
  TASK_DATA = results.load_results(path=os.path.join(bigbench_path,'benchmark_tasks'))
  ALL_TASKS = sorted(TASK_DATA.keys())
  #@title Select Task, Subtask, and Metric
  print("ALL_TASKS: ", len(ALL_TASKS), ALL_TASKS)

  ALL_TASKS = [_ for _ in ALL_TASKS if _ != 'training_on_test_set']

  #ALL_TASKS = ALL_TASKS[-90:]
  #ALL_TASKS = ALL_TASKS[:-90]

  print("ALL_TASKS: ", len(ALL_TASKS), ALL_TASKS)

  all_eval_metrics = []

  header = ['task_name', 'model_family', 'metric', 'shots', 'func_form', 'num_points_for_fit', 'downstream_performance', 'mse_train', 'msle_train', 'mse_all', 'msle_all', 'se_extrap', 'sle_extrap', 'a', 'b', 'c', 'd', 'f', 'g', 'NaN']
  with open('data.csv', 'a', encoding='UTF8') as f:
      writer = csv.writer(f)
      writer.writerow(header)

      for t in ALL_TASKS:
          initial_task = t
          initial_subtask = initial_task
          initial_metric = TASK_DATA[initial_task].preferred_metric[initial_subtask]

          task_dropdown = ipywidgets.Dropdown(
              options=ALL_TASKS, value=initial_task, description='Task')

          subtask_dropdown = ipywidgets.Dropdown(
              options=TASK_DATA[initial_task].subtasks,
              value=initial_subtask,
              description='Subtask')

          metric_dropdown = ipywidgets.Dropdown(
              options=sorted(TASK_DATA[initial_task].data[initial_subtask].keys()),
              value=initial_metric,
              description='Metric')

          normalize_multiple_choice_checkbox = ipywidgets.Checkbox(
              value=False,
              description='Normalize multiple_choice_grade scores',
              indent=False,
          )

          normalize_multiple_choice_label = ipywidgets.Label(
              value="Select the following checkbox to normalize multiple_choice_grade "
              "scores when the task's preferred score is multiple_choice_grade.")

          normalize_multiple_choice_label2 = ipywidgets.Label(
              value="This will "
              "make 0=random performance, and thus negative values may occur.")

          def task_update(*args):
              subtask_dropdown.options = TASK_DATA[task_dropdown.value].subtasks
              if task_dropdown.value in subtask_dropdown.options:
                  subtask_dropdown.value = task_dropdown.value
              else:
                  subtask_dropdown.value = subtask_dropdown.options[0]

          def subtask_update(*args):
              metrics_list = sorted(
                  TASK_DATA[task_dropdown.value].data[subtask_dropdown.value].keys())
              preferred_metric = TASK_DATA[task_dropdown.value].preferred_metric[subtask_dropdown.value]
              if preferred_metric in metrics_list:
                  metrics_list.remove(preferred_metric)
                  metrics_list = [preferred_metric] + metrics_list
              metric_dropdown.options = metrics_list

          task_dropdown.observe(task_update, 'value')
          subtask_dropdown.observe(subtask_update, 'value')

          #@title Plot
          plt_task = task_dropdown.value
          plt_subtask = subtask_dropdown.value
          plt_metric = metric_dropdown.value
          normalize_mc_grade = normalize_multiple_choice_checkbox.value

          model_families = ['BIG-G sparse']
          model_family = model_families[0]
          try:
            shots = TASK_DATA[plt_task].data[plt_subtask][plt_metric][model_family].keys()
          except:
            continue

          for _s in shots:
              data = plot_task(task_data=TASK_DATA[plt_task], subtask=plt_subtask, metric=plt_metric,
                      model_families=model_families,
                      shots=[_s],
                      plot_settings=PLOT_SETTINGS,
                      normalize_mc_grade=normalize_mc_grade)

  df = pd.DataFrame(data_dict)
  df.to_csv("error.csv")

  !zip -r plots.zip plots/


In [ ]:
#!zip -r plots.zip plots/

In [ ]:
#df = pd.DataFrame(data_dict)
#df.to_csv("error.csv")